In [1]:
from model_inference import llm

In [2]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor

def book_hotel(hotel_name: str):
    """Book a hotel"""
    return f"Successfully booked a stay at {hotel_name}."

def book_flight(from_airport: str, to_airport: str):
    """Book a flight"""
    return f"Successfully booked a flight from {from_airport} to {to_airport}."

flight_assistant = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=[book_flight],
    prompt="You are a flight booking assistant",
    name="flight_assistant"
)

hotel_assistant = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=[book_hotel],
    prompt="You are a hotel booking assistant",
    name="hotel_assistant"
)

supervisor = create_supervisor(
    agents=[flight_assistant, hotel_assistant],
    model=ChatOpenAI(model="gpt-4o-mini"),
    prompt=(
        "You manage a hotel booking assistant and a"
        "flight booking assistant. Assign work to them."
    )
).compile()


output = []
for chunk in supervisor.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "book a flight from BOS to JFK and a stay at McKittrick Hotel"
            }
        ]
    }
):
    print(chunk)
    output.append(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='book a flight from BOS to JFK and a stay at McKittrick Hotel', additional_kwargs={}, response_metadata={}, id='56fc4ec5-f8bd-438d-929e-1dade9647344'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_iuf8tYBw2iYgG1Ot2k4lYWIP', 'function': {'arguments': '{}', 'name': 'transfer_to_flight_assistant'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 100, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiHitg5XXfrrauujE6Uf1xlrIktSy', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--a9202948-45c8-4922-bb04-115a17f462e8-0', tool_ca

In [3]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, FunctionMessage
from pprint import pprint

response = output[-1]['supervisor']
# Iterate through the list of messages
for i, message in enumerate(response["messages"]):
    print(f"\nMessage {i + 1} - Type: {type(message).__name__}")
    print("-" * 50)

    # Handle HumanMessage
    if isinstance(message, HumanMessage):
        print(f"User Input: {message.content}")

    # Handle AIMessage
    elif isinstance(message, AIMessage):
        print(f"AI Response: {message.content}")
        if message.tool_calls:
            print("Tool Calls:")
            for tool_call in message.tool_calls:
                print(f"  Tool Name: {tool_call['name']}")
                print("  Arguments:")
                pprint(tool_call['args'])

    # Handle ToolMessage
    elif isinstance(message, ToolMessage):
        print(f"Tool Response from {message.tool_call_id}:")
        print(f"  Content: {message.content}")

    # Handle FunctionMessage
    elif isinstance(message, FunctionMessage):
        print("Function Response:")
        print(f"  Name: {message.name}")
        print(f"  Content: {message.content}")

    # Handle any other message type
    else:
        print("Other Message:")
        pprint(message)

    print("-*-" * 60)



Message 1 - Type: HumanMessage
--------------------------------------------------
User Input: book a flight from BOS to JFK and a stay at McKittrick Hotel
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-

Message 2 - Type: AIMessage
--------------------------------------------------
AI Response: 
Tool Calls:
  Tool Name: transfer_to_flight_assistant
  Arguments:
{}
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-

Message 3 - Type: ToolMessage
--------------------------------------------------
Tool Response from call_iuf8tYBw2iYgG1Ot2k4lYWIP:
  Content: Successfully transferred to flight_assistant
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--